In [6]:
import zipfile
import glob
import os

# Define where to put the final JPGs
extract_destination = "all_my_jpgs"

# Create the destination folder if it doesn't exist
os.makedirs(extract_destination, exist_ok=True)

# 1. Find all files ending in .zip in the current directory
zip_files = glob.glob('*.zip')

print(f"Found zip files: {zip_files}")

# 2. Loop through each zip file
for zip_path in zip_files:
    print(f"--- Opening: {zip_path} ---")

    # Open the current zip file in 'read' mode
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:

        # 3. Loop through every file inside the zip
        for file_path_in_zip in zip_ref.namelist():

            # 4. Check if the file is a JPG
            if file_path_in_zip.lower().endswith('.jpg'):
                print(f"  Found & Extracting: {file_path_in_zip}")

                # Extract this single JPG file to our destination
                zip_ref.extract(file_path_in_zip, path=extract_destination)

print(f"\nDone! All JPGs extracted to '{extract_destination}'.")

Found zip files: ['jurassic-world-20251020T105747Z-1-001.zip', 'star-wars-20251020T105752Z-1-001.zip', 'test-20251020T105831Z-1-001.zip', 'marvel-20251020T105748Z-1-001.zip']
--- Opening: jurassic-world-20251020T105747Z-1-001.zip ---
  Found & Extracting: jurassic-world/0001/004.jpg
  Found & Extracting: jurassic-world/0001/009.jpg
  Found & Extracting: jurassic-world/0001/007.jpg
  Found & Extracting: jurassic-world/0001/012.jpg
  Found & Extracting: jurassic-world/0002/013.jpg
  Found & Extracting: jurassic-world/0001/006.jpg
  Found & Extracting: jurassic-world/0001/005.jpg
  Found & Extracting: jurassic-world/0002/006.jpg
  Found & Extracting: jurassic-world/0001/001.jpg
  Found & Extracting: jurassic-world/0001/003.jpg
  Found & Extracting: jurassic-world/0002/012.jpg
  Found & Extracting: jurassic-world/0002/003.jpg
  Found & Extracting: jurassic-world/0002/005.jpg
  Found & Extracting: jurassic-world/0001/014.jpg
  Found & Extracting: jurassic-world/0002/004.jpg
  Found & Extrac

In [7]:
import tensorflow as tf

In [8]:
import tensorflow as tf
from tensorflow.keras import layers

# --- 1. Define Parameters ---
data_dir = "all_my_jpgs"  # The folder containing your class sub-folders
img_height = 128
img_width = 128
batch_size = 32

# --- 2. Load and Resize Data (Tasks 1 & 2) ---

# Create a training dataset (80% of data)
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,  # Use 20% of images for validation
  subset="training",
  seed=123,
  image_size=(img_height, img_width), # Resize to 128x128
  batch_size=batch_size
)

# Create a validation dataset (20% of data)
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

print(f"Class names: {train_ds.class_names}")

for images, labels in train_ds.take(1):
    print(f"\nImages batch shape: {images.shape}")
    print(f"Labels batch shape: {labels.shape}")
    print("--------------------")
    print(f"Pixel values (min): {images.numpy().min()}")
    print(f"Pixel values (max): {images.numpy().max()}")

Found 468 files belonging to 4 classes.
Using 375 files for training.
Found 468 files belonging to 4 classes.
Using 93 files for validation.
Class names: ['jurassic-world', 'marvel', 'star-wars', 'test']

Images batch shape: (32, 128, 128, 3)
Labels batch shape: (32,)
--------------------
Pixel values (min): 0.0
Pixel values (max): 255.0


In [9]:
from tensorflow.keras.models import Sequential


data_augmentation = Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
  ],
  name="data_augmentation"
)

normalization_layer = layers.Rescaling(1./255)

In [21]:
tf.keras.backend.clear_session()

In [22]:
num_classes = len(train_ds.class_names)
model = Sequential([
  layers.Input(shape=(img_height, img_width, 3)),
  normalization_layer,
  data_augmentation,
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     8,388,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,466,628 (32.30 MB)

 Trainable params: 8,466,628 (32.30 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history=model.fit(train_ds,epochs=30,batch_size=32,validation_data=val_ds)

Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 129ms/step - accuracy: 0.4017 - loss: 3.4799 - val_accuracy: 0.3978 - val_loss: 1.2371
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.4003 - loss: 1.2302 - val_accuracy: 0.3978 - val_loss: 1.1809
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.4481 - loss: 1.1843 - val_accuracy: 0.4946 - val_loss: 1.1734
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.4322 - loss: 1.2105 - val_accuracy: 0.4839 - val_loss: 1.1421
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.5077 - loss: 1.1589 - val_accuracy: 0.5054 - val_loss: 1.1229
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.4530 - loss: 1.1563 - val_accuracy: 0.4731 - val_loss: 1.1131
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.4925 - loss: 1.1283 - val_accuracy: 0.4409 - val_loss: 1.0902
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.4811 - loss: 1.0899 - val_accuracy: 0.4409 - 

In [25]:
# Evaluate the model
val_loss, vol_acc = model.evaluate(val_ds)
print("Test Loss:", val_loss)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6565 - loss: 0.8548
Test Loss: 0.9343030452728271
